In [1]:
import fastai
from fastai.vision import *
fastai.__version__
#import torch.nn.functional as F

'1.0.51'

In [2]:
Incepres_BATCH  = 36
Incepres_SIZE   = 320
path = Path('../input/imet-2019-fgvc6/') # iMet data path

In [3]:
Res_BATCH  = 36
Res_SIZE   = 320

In [4]:
Sex_BATCH  = 32
Sex_SIZE   = 256

In [5]:

from torch.utils import model_zoo
Path('models').mkdir(exist_ok=True)

In [6]:
train_df = pd.read_csv(path/'train.csv')
train_df.head()

,id,attribute_ids
0,1000483014d91860,147 616 813
1,1000fe2e667721fe,51 616 734 813
2,1001614cb89646ee,776
3,10041eb49b297c08,51 671 698 813 1092
4,100501c227f8beea,13 404 492 903 1093


In [7]:

labels_df = pd.read_csv(path/'labels.csv')
labels_df.head()

,attribute_id,attribute_name
0,0,culture::abruzzi
1,1,culture::achaemenid
2,2,culture::aegean
3,3,culture::afghan
4,4,culture::after british


In [8]:
test_df = pd.read_csv(path/'sample_submission.csv')
test_df.head()

,id,attribute_ids
0,10023b2cc4ed5f68,0 1 2
1,100fbe75ed8fd887,0 1 2
2,101b627524a04f19,0 1 2
3,10234480c41284c6,0 1 2
4,1023b0e2636dcea8,0 1 2


In [9]:


tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=0.10, max_zoom=1.5, max_warp=0.2, max_lighting=0.2,
                     xtra_tfms=[(symmetric_warp(magnitude=(-0,0), p=0)),])

In [10]:
train, test = [ImageList.from_df(df, path=path, cols='id', folder=folder, suffix='.png') 
               for df, folder in zip([train_df, test_df], ['train', 'test'])]
Incepres_data = (train.split_by_rand_pct(0.1, seed=42)
        .label_from_df(cols='attribute_ids', label_delim=' ')
        .add_test(test)
        .transform(tfms, size=Incepres_SIZE, resize_method=ResizeMethod.PAD, padding_mode='border',)
        .databunch(path=Path('.'), bs=Incepres_BATCH).normalize(imagenet_stats))
Res_data = (train.split_by_rand_pct(0.1, seed=42)
        .label_from_df(cols='attribute_ids', label_delim=' ')
        .add_test(test)
        .transform(tfms, size=Res_SIZE, resize_method=ResizeMethod.PAD, padding_mode='border',)
        .databunch(path=Path('.'), bs=Res_BATCH).normalize(imagenet_stats))
Sex_data = (train.split_by_rand_pct(0.1, seed=42)
        .label_from_df(cols='attribute_ids', label_delim=' ')
        .add_test(test)
        .transform(tfms, size=Sex_SIZE, resize_method=ResizeMethod.PAD, padding_mode='border',)
        .databunch(path=Path('.'), bs=Sex_BATCH).normalize(imagenet_stats))

In [11]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma

    def forward(self, logit, target):
        target = target.float()
        max_val = (-logit).clamp(min=0)
        loss = logit - logit * target + max_val + \
               ((-max_val).exp() + (-logit - max_val).exp()).log()

        invprobs = F.logsigmoid(-logit * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        if len(loss.size())==2:
            loss = loss.sum(dim=1)
        return loss.mean()

In [12]:
"""__all__ = ['inceptionv4', 'inceptionresnetv2', 'nasnetamobile', 'dpn92', 'xception_cadene', 'se_resnet50',

           'se_resnet101', 'se_resnext50_32x4d', 'senet154', 'pnasnet5large']



def get_model(model_name:str, pretrained:bool, seq:bool=False, pname:str='imagenet', **kwargs):

    pretrained = pname if pretrained else None

    model = getattr(pretrainedmodels, model_name)(pretrained=pretrained, **kwargs)

    return nn.Sequential(*model.children()) if seq else model"""

"__all__ = ['inceptionv4', 'inceptionresnetv2', 'nasnetamobile', 'dpn92', 'xception_cadene', 'se_resnet50',\n\n           'se_resnet101', 'se_resnext50_32x4d', 'senet154', 'pnasnet5large']\n\n\n\ndef get_model(model_name:str, pretrained:bool, seq:bool=False, pname:str='imagenet', **kwargs):\n\n    pretrained = pname if pretrained else None\n\n    model = getattr(pretrainedmodels, model_name)(pretrained=pretrained, **kwargs)\n\n    return nn.Sequential(*model.children()) if seq else model"

In [13]:
from fastai.vision import learner


In [14]:
from __future__ import print_function, division, absolute_import

import torch

import torch.nn as nn

import torch.utils.model_zoo as model_zoo

import os

import sys



__all__ = ['InceptionResNetV2', 'inceptionresnetv2']



pretrained_settings = {

    'inceptionresnetv2': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth',

            'input_space': 'RGB',

            'input_size': [3, 299, 299],

            'input_range': [0, 1],

            'mean': [0.5, 0.5, 0.5],

            'std': [0.5, 0.5, 0.5],

            'num_classes': 1000

        },

        'imagenet+background': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth',

            'input_space': 'RGB',

            'input_size': [3, 299, 299],

            'input_range': [0, 1],

            'mean': [0.5, 0.5, 0.5],

            'std': [0.5, 0.5, 0.5],

            'num_classes': 1001

        }

    }

}





class BasicConv2d(nn.Module):



    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):

        super(BasicConv2d, self).__init__()

        self.conv = nn.Conv2d(in_planes, out_planes,

                              kernel_size=kernel_size, stride=stride,

                              padding=padding, bias=False) # verify bias false

        self.bn = nn.BatchNorm2d(out_planes,

                                 eps=0.001, # value found in tensorflow

                                 momentum=0.1, # default pytorch value

                                 affine=True)

        self.relu = nn.ReLU(inplace=False)



    def forward(self, x):

        x = self.conv(x)

        x = self.bn(x)

        x = self.relu(x)

        return x





class Mixed_5b(nn.Module):



    def __init__(self):

        super(Mixed_5b, self).__init__()



        self.branch0 = BasicConv2d(192, 96, kernel_size=1, stride=1)



        self.branch1 = nn.Sequential(

            BasicConv2d(192, 48, kernel_size=1, stride=1),

            BasicConv2d(48, 64, kernel_size=5, stride=1, padding=2)

        )



        self.branch2 = nn.Sequential(

            BasicConv2d(192, 64, kernel_size=1, stride=1),

            BasicConv2d(64, 96, kernel_size=3, stride=1, padding=1),

            BasicConv2d(96, 96, kernel_size=3, stride=1, padding=1)

        )



        self.branch3 = nn.Sequential(

            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),

            BasicConv2d(192, 64, kernel_size=1, stride=1)

        )



    def forward(self, x):

        x0 = self.branch0(x)

        x1 = self.branch1(x)

        x2 = self.branch2(x)

        x3 = self.branch3(x)

        out = torch.cat((x0, x1, x2, x3), 1)

        return out





class Block35(nn.Module):



    def __init__(self, scale=1.0):

        super(Block35, self).__init__()



        self.scale = scale



        self.branch0 = BasicConv2d(320, 32, kernel_size=1, stride=1)



        self.branch1 = nn.Sequential(

            BasicConv2d(320, 32, kernel_size=1, stride=1),

            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1)

        )



        self.branch2 = nn.Sequential(

            BasicConv2d(320, 32, kernel_size=1, stride=1),

            BasicConv2d(32, 48, kernel_size=3, stride=1, padding=1),

            BasicConv2d(48, 64, kernel_size=3, stride=1, padding=1)

        )



        self.conv2d = nn.Conv2d(128, 320, kernel_size=1, stride=1)

        self.relu = nn.ReLU(inplace=False)



    def forward(self, x):

        x0 = self.branch0(x)

        x1 = self.branch1(x)

        x2 = self.branch2(x)

        out = torch.cat((x0, x1, x2), 1)

        out = self.conv2d(out)

        out = out * self.scale + x

        out = self.relu(out)

        return out





class Mixed_6a(nn.Module):



    def __init__(self):

        super(Mixed_6a, self).__init__()



        self.branch0 = BasicConv2d(320, 384, kernel_size=3, stride=2)



        self.branch1 = nn.Sequential(

            BasicConv2d(320, 256, kernel_size=1, stride=1),

            BasicConv2d(256, 256, kernel_size=3, stride=1, padding=1),

            BasicConv2d(256, 384, kernel_size=3, stride=2)

        )



        self.branch2 = nn.MaxPool2d(3, stride=2)



    def forward(self, x):

        x0 = self.branch0(x)

        x1 = self.branch1(x)

        x2 = self.branch2(x)

        out = torch.cat((x0, x1, x2), 1)

        return out





class Block17(nn.Module):



    def __init__(self, scale=1.0):

        super(Block17, self).__init__()



        self.scale = scale



        self.branch0 = BasicConv2d(1088, 192, kernel_size=1, stride=1)



        self.branch1 = nn.Sequential(

            BasicConv2d(1088, 128, kernel_size=1, stride=1),

            BasicConv2d(128, 160, kernel_size=(1,7), stride=1, padding=(0,3)),

            BasicConv2d(160, 192, kernel_size=(7,1), stride=1, padding=(3,0))

        )



        self.conv2d = nn.Conv2d(384, 1088, kernel_size=1, stride=1)

        self.relu = nn.ReLU(inplace=False)



    def forward(self, x):

        x0 = self.branch0(x)

        x1 = self.branch1(x)

        out = torch.cat((x0, x1), 1)

        out = self.conv2d(out)

        out = out * self.scale + x

        out = self.relu(out)

        return out





class Mixed_7a(nn.Module):



    def __init__(self):

        super(Mixed_7a, self).__init__()



        self.branch0 = nn.Sequential(

            BasicConv2d(1088, 256, kernel_size=1, stride=1),

            BasicConv2d(256, 384, kernel_size=3, stride=2)

        )



        self.branch1 = nn.Sequential(

            BasicConv2d(1088, 256, kernel_size=1, stride=1),

            BasicConv2d(256, 288, kernel_size=3, stride=2)

        )



        self.branch2 = nn.Sequential(

            BasicConv2d(1088, 256, kernel_size=1, stride=1),

            BasicConv2d(256, 288, kernel_size=3, stride=1, padding=1),

            BasicConv2d(288, 320, kernel_size=3, stride=2)

        )



        self.branch3 = nn.MaxPool2d(3, stride=2)



    def forward(self, x):

        x0 = self.branch0(x)

        x1 = self.branch1(x)

        x2 = self.branch2(x)

        x3 = self.branch3(x)

        out = torch.cat((x0, x1, x2, x3), 1)

        return out





class Block8(nn.Module):



    def __init__(self, scale=1.0, noReLU=False):

        super(Block8, self).__init__()



        self.scale = scale

        self.noReLU = noReLU



        self.branch0 = BasicConv2d(2080, 192, kernel_size=1, stride=1)



        self.branch1 = nn.Sequential(

            BasicConv2d(2080, 192, kernel_size=1, stride=1),

            BasicConv2d(192, 224, kernel_size=(1,3), stride=1, padding=(0,1)),

            BasicConv2d(224, 256, kernel_size=(3,1), stride=1, padding=(1,0))

        )



        self.conv2d = nn.Conv2d(448, 2080, kernel_size=1, stride=1)

        if not self.noReLU:

            self.relu = nn.ReLU(inplace=False)



    def forward(self, x):

        x0 = self.branch0(x)

        x1 = self.branch1(x)

        out = torch.cat((x0, x1), 1)

        out = self.conv2d(out)

        out = out * self.scale + x

        if not self.noReLU:

            out = self.relu(out)

        return out





class InceptionResNetV2(nn.Module):



    def __init__(self, num_classes=1001):

        super(InceptionResNetV2, self).__init__()

        # Special attributs

        self.input_space = None

        self.input_size = (299, 299, 3)

        self.mean = None

        self.std = None

        # Modules

        self.conv2d_1a = BasicConv2d(3, 32, kernel_size=3, stride=2)

        self.conv2d_2a = BasicConv2d(32, 32, kernel_size=3, stride=1)

        self.conv2d_2b = BasicConv2d(32, 64, kernel_size=3, stride=1, padding=1)

        self.maxpool_3a = nn.MaxPool2d(3, stride=2)

        self.conv2d_3b = BasicConv2d(64, 80, kernel_size=1, stride=1)

        self.conv2d_4a = BasicConv2d(80, 192, kernel_size=3, stride=1)

        self.maxpool_5a = nn.MaxPool2d(3, stride=2)

        self.mixed_5b = Mixed_5b()

        self.repeat = nn.Sequential(

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17),

            Block35(scale=0.17)

        )

        self.mixed_6a = Mixed_6a()

        self.repeat_1 = nn.Sequential(

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10),

            Block17(scale=0.10)

        )

        self.mixed_7a = Mixed_7a()

        self.repeat_2 = nn.Sequential(

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20),

            Block8(scale=0.20)

        )

        self.block8 = Block8(noReLU=True)

        self.conv2d_7b = BasicConv2d(2080, 1536, kernel_size=1, stride=1)

        self.avgpool_1a = nn.AvgPool2d(8, count_include_pad=False)

        self.last_linear = nn.Linear(1536, num_classes)



    def features(self, input):

        x = self.conv2d_1a(input)

        x = self.conv2d_2a(x)

        x = self.conv2d_2b(x)

        x = self.maxpool_3a(x)

        x = self.conv2d_3b(x)

        x = self.conv2d_4a(x)

        x = self.maxpool_5a(x)

        x = self.mixed_5b(x)

        x = self.repeat(x)

        x = self.mixed_6a(x)

        x = self.repeat_1(x)

        x = self.mixed_7a(x)

        x = self.repeat_2(x)

        x = self.block8(x)

        x = self.conv2d_7b(x)

        return x



    def logits(self, features):

        x = self.avgpool_1a(features)

        x = x.view(x.size(0), -1)

        x = self.last_linear(x)

        return x



    def forward(self, input):

        x = self.features(input)

        x = self.logits(x)

        return x



def inceptionresnetv2(num_classes=1000, pretrained='imagenet'):

    r"""InceptionResNetV2 model architecture from the

    `"InceptionV4, Inception-ResNet..." <https://arxiv.org/abs/1602.07261>`_ paper.

    """

    if pretrained:

        settings = pretrained_settings['inceptionresnetv2'][pretrained]

        assert num_classes == settings['num_classes'],"num_classes should be {}, but is {}".format(settings['num_classes'], num_classes)



        # both 'imagenet'&'imagenet+background' are loaded from same parameters

        model = InceptionResNetV2(num_classes=1001)

        model.load_state_dict(model_zoo.load_url(settings['url']))



        if pretrained == 'imagenet':

            new_last_linear = nn.Linear(1536, 1000)

            new_last_linear.weight.data = model.last_linear.weight.data[1:]

            new_last_linear.bias.data = model.last_linear.bias.data[1:]

            model.last_linear = new_last_linear



        model.input_space = settings['input_space']

        model.input_size = settings['input_size']

        model.input_range = settings['input_range']



        model.mean = settings['mean']

        model.std = settings['std']

    else:

        model = InceptionResNetV2(num_classes=num_classes)

    return model



'''

TEST

Run this code with:

```

cd $HOME/pretrained-models.pytorch

python -m pretrainedmodels.inceptionresnetv2

```

'''

#if __name__ == '__main__':



 #   assert inceptionresnetv2(num_classes=10, pretrained=None)

  #  print('success')

   # assert inceptionresnetv2(num_classes=1000, pretrained='imagenet')

    #print('success')

   # assert inceptionresnetv2(num_classes=1001, pretrained='imagenet+background')

    #print('success')



    # fail

   # assert inceptionresnetv2(num_classes=1001, pretrained='imagenet')

'\n\nTEST\n\nRun this code with:\n\n```\n\ncd $HOME/pretrained-models.pytorch\n\npython -m pretrainedmodels.inceptionresnetv2\n\n```\n\n'

In [15]:
from __future__ import print_function, division, absolute_import

from collections import OrderedDict

import math



import torch.nn as nn

from torch.utils import model_zoo



__all__ = ['SENet', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152',

           'se_resnext50_32x4d', 'se_resnext101_32x4d']



pretrained_settings = {

    'senet154': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/senet154-c7b49a05.pth',

            'input_space': 'RGB',

            'input_size': [3, 224, 224],

            'input_range': [0, 1],

            'mean': [0.485, 0.456, 0.406],

            'std': [0.229, 0.224, 0.225],

            'num_classes': 1000

        }

    },

    'se_resnet50': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet50-ce0d4300.pth',

            'input_space': 'RGB',

            'input_size': [3, 224, 224],

            'input_range': [0, 1],

            'mean': [0.485, 0.456, 0.406],

            'std': [0.229, 0.224, 0.225],

            'num_classes': 1000

        }

    },

    'se_resnet101': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet101-7e38fcc6.pth',

            'input_space': 'RGB',

            'input_size': [3, 224, 224],

            'input_range': [0, 1],

            'mean': [0.485, 0.456, 0.406],

            'std': [0.229, 0.224, 0.225],

            'num_classes': 1000

        }

    },

    'se_resnet152': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet152-d17c99b7.pth',

            'input_space': 'RGB',

            'input_size': [3, 224, 224],

            'input_range': [0, 1],

            'mean': [0.485, 0.456, 0.406],

            'std': [0.229, 0.224, 0.225],

            'num_classes': 1000

        }

    },

    'se_resnext50_32x4d': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth',

            'input_space': 'RGB',

            'input_size': [3, 224, 224],

            'input_range': [0, 1],

            'mean': [0.485, 0.456, 0.406],

            'std': [0.229, 0.224, 0.225],

            'num_classes': 1000

        }

    },

    'se_resnext101_32x4d': {

        'imagenet': {

            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth',

            'input_space': 'RGB',

            'input_size': [3, 224, 224],

            'input_range': [0, 1],

            'mean': [0.485, 0.456, 0.406],

            'std': [0.229, 0.224, 0.225],

            'num_classes': 1000

        }

    },

}





class SEModule(nn.Module):



    def __init__(self, channels, reduction):

        super(SEModule, self).__init__()

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,

                             padding=0)

        self.relu = nn.ReLU(inplace=True)

        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,

                             padding=0)

        self.sigmoid = nn.Sigmoid()



    def forward(self, x):

        module_input = x

        x = self.avg_pool(x)

        x = self.fc1(x)

        x = self.relu(x)

        x = self.fc2(x)

        x = self.sigmoid(x)

        return module_input * x





class Bottleneck(nn.Module):

    """

    Base class for bottlenecks that implements `forward()` method.

    """

    def forward(self, x):

        residual = x



        out = self.conv1(x)

        out = self.bn1(out)

        out = self.relu(out)



        out = self.conv2(out)

        out = self.bn2(out)

        out = self.relu(out)



        out = self.conv3(out)

        out = self.bn3(out)



        if self.downsample is not None:

            residual = self.downsample(x)



        out = self.se_module(out) + residual

        out = self.relu(out)



        return out





class SEBottleneck(Bottleneck):

    """

    Bottleneck for SENet154.

    """

    expansion = 4



    def __init__(self, inplanes, planes, groups, reduction, stride=1,

                 downsample=None):

        super(SEBottleneck, self).__init__()

        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)

        self.bn1 = nn.BatchNorm2d(planes * 2)

        self.conv2 = nn.Conv2d(planes * 2, planes * 4, kernel_size=3,

                               stride=stride, padding=1, groups=groups,

                               bias=False)

        self.bn2 = nn.BatchNorm2d(planes * 4)

        self.conv3 = nn.Conv2d(planes * 4, planes * 4, kernel_size=1,

                               bias=False)

        self.bn3 = nn.BatchNorm2d(planes * 4)

        self.relu = nn.ReLU(inplace=True)

        self.se_module = SEModule(planes * 4, reduction=reduction)

        self.downsample = downsample

        self.stride = stride





class SEResNetBottleneck(Bottleneck):

    """

    ResNet bottleneck with a Squeeze-and-Excitation module. It follows Caffe

    implementation and uses `stride=stride` in `conv1` and not in `conv2`

    (the latter is used in the torchvision implementation of ResNet).

    """

    expansion = 4



    def __init__(self, inplanes, planes, groups, reduction, stride=1,

                 downsample=None):

        super(SEResNetBottleneck, self).__init__()

        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False,

                               stride=stride)

        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1,

                               groups=groups, bias=False)

        self.bn2 = nn.BatchNorm2d(planes)

        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)

        self.bn3 = nn.BatchNorm2d(planes * 4)

        self.relu = nn.ReLU(inplace=True)

        self.se_module = SEModule(planes * 4, reduction=reduction)

        self.downsample = downsample

        self.stride = stride





class SEResNeXtBottleneck(Bottleneck):

    """

    ResNeXt bottleneck type C with a Squeeze-and-Excitation module.

    """

    expansion = 4



    def __init__(self, inplanes, planes, groups, reduction, stride=1,

                 downsample=None, base_width=4):

        super(SEResNeXtBottleneck, self).__init__()

        width = math.floor(planes * (base_width / 64)) * groups

        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False,

                               stride=1)

        self.bn1 = nn.BatchNorm2d(width)

        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride,

                               padding=1, groups=groups, bias=False)

        self.bn2 = nn.BatchNorm2d(width)

        self.conv3 = nn.Conv2d(width, planes * 4, kernel_size=1, bias=False)

        self.bn3 = nn.BatchNorm2d(planes * 4)

        self.relu = nn.ReLU(inplace=True)

        self.se_module = SEModule(planes * 4, reduction=reduction)

        self.downsample = downsample

        self.stride = stride





class SENet(nn.Module):



    def __init__(self, block, layers, groups, reduction, dropout_p=0.2,

                 inplanes=128, input_3x3=True, downsample_kernel_size=3,

                 downsample_padding=1, num_classes=1000):

        """

        Parameters

        ----------

        block (nn.Module): Bottleneck class.

            - For SENet154: SEBottleneck

            - For SE-ResNet models: SEResNetBottleneck

            - For SE-ResNeXt models:  SEResNeXtBottleneck

        layers (list of ints): Number of residual blocks for 4 layers of the

            network (layer1...layer4).

        groups (int): Number of groups for the 3x3 convolution in each

            bottleneck block.

            - For SENet154: 64

            - For SE-ResNet models: 1

            - For SE-ResNeXt models:  32

        reduction (int): Reduction ratio for Squeeze-and-Excitation modules.

            - For all models: 16

        dropout_p (float or None): Drop probability for the Dropout layer.

            If `None` the Dropout layer is not used.

            - For SENet154: 0.2

            - For SE-ResNet models: None

            - For SE-ResNeXt models: None

        inplanes (int):  Number of input channels for layer1.

            - For SENet154: 128

            - For SE-ResNet models: 64

            - For SE-ResNeXt models: 64

        input_3x3 (bool): If `True`, use three 3x3 convolutions instead of

            a single 7x7 convolution in layer0.

            - For SENet154: True

            - For SE-ResNet models: False

            - For SE-ResNeXt models: False

        downsample_kernel_size (int): Kernel size for downsampling convolutions

            in layer2, layer3 and layer4.

            - For SENet154: 3

            - For SE-ResNet models: 1

            - For SE-ResNeXt models: 1

        downsample_padding (int): Padding for downsampling convolutions in

            layer2, layer3 and layer4.

            - For SENet154: 1

            - For SE-ResNet models: 0

            - For SE-ResNeXt models: 0

        num_classes (int): Number of outputs in `last_linear` layer.

            - For all models: 1000

        """

        super(SENet, self).__init__()

        self.inplanes = inplanes

        if input_3x3:

            layer0_modules = [

                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1,

                                    bias=False)),

                ('bn1', nn.BatchNorm2d(64)),

                ('relu1', nn.ReLU(inplace=True)),

                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1,

                                    bias=False)),

                ('bn2', nn.BatchNorm2d(64)),

                ('relu2', nn.ReLU(inplace=True)),

                ('conv3', nn.Conv2d(64, inplanes, 3, stride=1, padding=1,

                                    bias=False)),

                ('bn3', nn.BatchNorm2d(inplanes)),

                ('relu3', nn.ReLU(inplace=True)),

            ]

        else:

            layer0_modules = [

                ('conv1', nn.Conv2d(3, inplanes, kernel_size=7, stride=2,

                                    padding=3, bias=False)),

                ('bn1', nn.BatchNorm2d(inplanes)),

                ('relu1', nn.ReLU(inplace=True)),

            ]

        # To preserve compatibility with Caffe weights `ceil_mode=True`

        # is used instead of `padding=1`.

        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2,

                                                    ceil_mode=True)))

        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))

        self.layer1 = self._make_layer(

            block,

            planes=64,

            blocks=layers[0],

            groups=groups,

            reduction=reduction,

            downsample_kernel_size=1,

            downsample_padding=0

        )

        self.layer2 = self._make_layer(

            block,

            planes=128,

            blocks=layers[1],

            stride=2,

            groups=groups,

            reduction=reduction,

            downsample_kernel_size=downsample_kernel_size,

            downsample_padding=downsample_padding

        )

        self.layer3 = self._make_layer(

            block,

            planes=256,

            blocks=layers[2],

            stride=2,

            groups=groups,

            reduction=reduction,

            downsample_kernel_size=downsample_kernel_size,

            downsample_padding=downsample_padding

        )

        self.layer4 = self._make_layer(

            block,

            planes=512,

            blocks=layers[3],

            stride=2,

            groups=groups,

            reduction=reduction,

            downsample_kernel_size=downsample_kernel_size,

            downsample_padding=downsample_padding

        )

        self.avg_pool = nn.AvgPool2d(7, stride=1)

        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None

        self.last_linear = nn.Linear(512 * block.expansion, num_classes)



    def _make_layer(self, block, planes, blocks, groups, reduction, stride=1,

                    downsample_kernel_size=1, downsample_padding=0):

        downsample = None

        if stride != 1 or self.inplanes != planes * block.expansion:

            downsample = nn.Sequential(

                nn.Conv2d(self.inplanes, planes * block.expansion,

                          kernel_size=downsample_kernel_size, stride=stride,

                          padding=downsample_padding, bias=False),

                nn.BatchNorm2d(planes * block.expansion),

            )



        layers = []

        layers.append(block(self.inplanes, planes, groups, reduction, stride,

                            downsample))

        self.inplanes = planes * block.expansion

        for i in range(1, blocks):

            layers.append(block(self.inplanes, planes, groups, reduction))



        return nn.Sequential(*layers)



    def features(self, x):

        x = self.layer0(x)

        x = self.layer1(x)

        x = self.layer2(x)

        x = self.layer3(x)

        x = self.layer4(x)

        return x



    def logits(self, x):

        x = self.avg_pool(x)

        if self.dropout is not None:

            x = self.dropout(x)

        x = x.view(x.size(0), -1)

        x = self.last_linear(x)

        return x



    def forward(self, x):

        x = self.features(x)

        x = self.logits(x)

        return x





def initialize_pretrained_model(model, num_classes, settings):

    assert num_classes == settings['num_classes'], 'num_classes should be {}, but is {}'.format(

            settings['num_classes'], num_classes)

    model.load_state_dict(model_zoo.load_url(settings['url']))

    model.input_space = settings['input_space']

    model.input_size = settings['input_size']

    model.input_range = settings['input_range']

    model.mean = settings['mean']

    model.std = settings['std']





def senet154(num_classes=1000, pretrained='imagenet'):

    model = SENet(SEBottleneck, [3, 8, 36, 3], groups=64, reduction=16,

                  dropout_p=0.2, num_classes=num_classes)

    if pretrained is not None:

        settings = pretrained_settings['senet154'][pretrained]

        initialize_pretrained_model(model, num_classes, settings)

    return model





def se_resnet50(num_classes=1000, pretrained='imagenet'):

    model = SENet(SEResNetBottleneck, [3, 4, 6, 3], groups=1, reduction=16,

                  dropout_p=None, inplanes=64, input_3x3=False,

                  downsample_kernel_size=1, downsample_padding=0,

                  num_classes=num_classes)

    if pretrained is not None:

        settings = pretrained_settings['se_resnet50'][pretrained]

        initialize_pretrained_model(model, num_classes, settings)

    return model





def se_resnet101(num_classes=1000, pretrained='imagenet'):

    model = SENet(SEResNetBottleneck, [3, 4, 23, 3], groups=1, reduction=16,

                  dropout_p=None, inplanes=64, input_3x3=False,

                  downsample_kernel_size=1, downsample_padding=0,

                  num_classes=num_classes)

    if pretrained is not None:

        settings = pretrained_settings['se_resnet101'][pretrained]

        initialize_pretrained_model(model, num_classes, settings)

    return model





def se_resnet152(num_classes=1000, pretrained='imagenet'):

    model = SENet(SEResNetBottleneck, [3, 8, 36, 3], groups=1, reduction=16,

                  dropout_p=None, inplanes=64, input_3x3=False,

                  downsample_kernel_size=1, downsample_padding=0,

                  num_classes=num_classes)

    if pretrained is not None:

        settings = pretrained_settings['se_resnet152'][pretrained]

        initialize_pretrained_model(model, num_classes, settings)

    return model





def se_resnext50_32x4d(num_classes=1000, pretrained='imagenet'):

    model = SENet(SEResNeXtBottleneck, [3, 4, 6, 3], groups=32, reduction=16,

                  dropout_p=None, inplanes=64, input_3x3=False,

                  downsample_kernel_size=1, downsample_padding=0,

                  num_classes=num_classes)

    if pretrained is not None:

        settings = pretrained_settings['se_resnext50_32x4d'][pretrained]

        initialize_pretrained_model(model, num_classes, settings)

    return model





def se_resnext101_32x4d(num_classes=1000, pretrained='imagenet'):

    model = SENet(SEResNeXtBottleneck, [3, 4, 23, 3], groups=32, reduction=16,

                  dropout_p=None, inplanes=64, input_3x3=False,

                  downsample_kernel_size=1, downsample_padding=0,

                  num_classes=num_classes)

    if pretrained is not None:

        settings = pretrained_settings['se_resnext101_32x4d'][pretrained]

        initialize_pretrained_model(model, num_classes, settings)

    return model

In [16]:
def get_incepres_model(model_name:str, pretrained:bool, seq:bool=False, pname:str='imagenet', **kwargs):

    pretrained = pname if pretrained else None

    model = inceptionresnetv2(pretrained=pretrained, **kwargs)

    return nn.Sequential(*model.children()) if seq else model

def get_sex_model(model_name:str, pretrained:bool, seq:bool=False, pname:str='imagenet', **kwargs):

    pretrained = pname if pretrained else None

    model = se_resnext50_32x4d(pretrained=pretrained, **kwargs)

    return nn.Sequential(*model.children()) if seq else model

In [17]:
def myinceptionresnetv2(pretrained:bool=False):  
    return get_incepres_model('inceptionresnetv2', pretrained, seq=True)
def myse_resnext50_32x4d(pretrained:bool=False): return get_sex_model('se_resnext50_32x4d', pretrained)
_se_resnet_meta               = {'cut': -2, 'split': lambda m: (m[0][3],     m[1])}
learner.model_meta[myinceptionresnetv2] = {'cut': -2, 'split': lambda m: (m[0][9],     m[1])}
learner.model_meta[se_resnext50_32x4d] = _se_resnet_meta

In [18]:
#Sex0
!cp ../input/seresnextkfolds/seresnext-11.pth ./models/
Sex_learn0 = cnn_learner(Sex_data, base_arch=myse_resnext50_32x4d, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Sex_learn0.load('seresnext-11')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Sex_test_preds0 = Sex_learn0.get_preds(ds_type=DatasetType.Test)
Sex_test_preds0


[tensor([[-4.2397, -4.0724, -4.7869,  ..., -3.6812, -4.1953, -3.3465],
         [-6.6699, -4.1217, -6.1984,  ..., -5.1072, -5.1527, -4.2143],
         [-4.4237, -2.8227, -4.8283,  ..., -3.7157, -3.5034, -3.5040],
         ...,
         [-3.8022, -3.6820, -4.7807,  ..., -4.7993, -4.0049, -3.7487],
         [-1.7127, -3.7101, -5.0781,  ..., -4.4098, -4.3867, -2.8275],
         [-6.3184, -4.5317, -5.4659,  ..., -5.5189, -5.2206, -4.3289]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [19]:
#Sex1
!cp ../input/seresnextkfolds/seresnext-15.pth ./models/
Sex_learn1 = cnn_learner(Sex_data, base_arch=myse_resnext50_32x4d, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Sex_learn1.load('seresnext-15')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Sex_test_preds1= Sex_learn1.get_preds(ds_type=DatasetType.Test)
Sex_test_preds1

[tensor([[-4.7257, -4.2600, -5.5239,  ..., -4.6999, -4.4859, -3.9938],
         [-5.9111, -3.4866, -6.1484,  ..., -4.8030, -4.3568, -3.8372],
         [-4.4627, -2.8329, -5.3345,  ..., -4.0522, -3.5207, -3.9738],
         ...,
         [-4.6109, -3.6496, -5.8393,  ..., -5.3187, -4.5054, -4.1572],
         [-1.9801, -4.5823, -5.6289,  ..., -5.0457, -4.8576, -3.0815],
         [-6.0796, -5.2004, -5.7849,  ..., -5.7858, -5.6564, -5.0043]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [20]:
#Sex0
!cp ../input/seresnextkfolds/seresnext-16.pth ./models/
Sex_learn2 = cnn_learner(Sex_data, base_arch=myse_resnext50_32x4d, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Sex_learn2.load('seresnext-16')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Sex_test_preds2 = Sex_learn2.get_preds(ds_type=DatasetType.Test)
Sex_test_preds2

[tensor([[-4.7402, -3.9168, -4.8009,  ..., -4.2831, -4.2637, -3.9519],
         [-4.5881, -3.0997, -4.6760,  ..., -4.6051, -4.0795, -3.2453],
         [-5.1202, -3.2000, -5.8567,  ..., -4.5047, -4.4605, -3.9419],
         ...,
         [-4.2696, -4.0256, -6.3115,  ..., -4.8854, -4.2005, -4.4948],
         [-1.7960, -3.7879, -5.6312,  ..., -4.9248, -4.6033, -2.8572],
         [-8.1181, -5.5333, -7.9278,  ..., -7.8838, -7.2008, -5.6765]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [21]:
#Sex0
!cp ../input/seresnextkfolds/seresnext-49.pth ./models/
Sex_learn3 = cnn_learner(Sex_data, base_arch=myse_resnext50_32x4d, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Sex_learn3.load('seresnext-49')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Sex_test_preds3 = Sex_learn3.get_preds(ds_type=DatasetType.Test)
Sex_test_preds3

[tensor([[-4.4752, -3.2928, -4.7780,  ..., -4.1319, -4.0518, -3.1248],
         [-5.4003, -3.4011, -5.4816,  ..., -5.2581, -4.8221, -3.8542],
         [-4.1364, -2.7257, -4.6923,  ..., -3.6289, -3.2891, -2.8974],
         ...,
         [-5.0552, -4.5161, -6.3873,  ..., -6.5459, -5.2020, -4.2509],
         [-2.2971, -4.3423, -5.8880,  ..., -5.4546, -4.6948, -3.2212],
         [-6.2667, -5.2960, -6.5377,  ..., -5.6132, -5.8716, -5.1003]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [22]:
#!ls ../input/15foldincepres/15foldincepres.pth

In [23]:


!ls ../input/incepres15fold320 


inceptionres-0-v4-52.pth   inceptionres-2-v4-52.pth    inceptionres-8-v4-60.pth
inceptionres-1-v4-54.pth   inceptionres-3-v4-49-n.pth
inceptionres-10-v4-48.pth  inceptionres-7-v4-60-n.pth


In [24]:
#cv0.593
!cp ../input/incepres15fold320/inceptionres-0-v4-52.pth ./models/
Incepres_learn0 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn0.load('inceptionres-0-v4-52')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds0 = Incepres_learn0.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds0

[tensor([[-4.8554, -4.5547, -4.9981,  ..., -4.7190, -4.1238, -4.5529],
         [-4.6653, -4.0573, -5.3674,  ..., -4.1696, -3.6554, -3.5170],
         [-5.3810, -4.0226, -6.0741,  ..., -5.1660, -4.6828, -4.4635],
         ...,
         [-3.2977, -3.2145, -4.0811,  ..., -3.0102, -2.6572, -2.8196],
         [-2.0332, -3.1249, -3.8445,  ..., -3.5523, -3.5527, -3.3104],
         [-6.3261, -5.0410, -6.6183,  ..., -5.7239, -5.7585, -5.0427]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [25]:
#cv0.597
!cp ../input/incepres15fold320/inceptionres-1-v4-54.pth ./models/
Incepres_learn1 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn1.load('inceptionres-1-v4-54')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds1 = Incepres_learn1.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds1

[tensor([[-4.0798, -3.6854, -4.3940,  ..., -3.2502, -3.3668, -3.7226],
         [-4.8521, -3.2104, -5.2860,  ..., -4.3698, -3.8726, -4.1253],
         [-4.5614, -3.0038, -4.7213,  ..., -3.8075, -3.7133, -3.7637],
         ...,
         [-3.9734, -4.1183, -5.2440,  ..., -4.5145, -3.5783, -3.7796],
         [-1.6109, -3.4681, -4.5964,  ..., -3.8445, -3.8284, -2.9832],
         [-5.4849, -5.1176, -6.1712,  ..., -5.2036, -4.7114, -4.6408]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [26]:
#cv0.588
!cp ../input/incepres15fold320/inceptionres-2-v4-52.pth ./models/
Incepres_learn2 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn2.load('inceptionres-2-v4-52')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds2 = Incepres_learn2.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds2

[tensor([[-5.2446, -3.8134, -4.3181,  ..., -4.5123, -4.0610, -3.7033],
         [-4.1951, -3.3780, -4.9933,  ..., -4.0850, -3.7152, -3.3751],
         [-5.0462, -3.1344, -5.8141,  ..., -5.2075, -4.3069, -4.3183],
         ...,
         [-3.6304, -3.6573, -4.8315,  ..., -4.3874, -3.6527, -3.8939],
         [-1.8335, -4.2679, -4.8562,  ..., -4.1590, -4.0741, -3.4824],
         [-7.2290, -5.6200, -7.3755,  ..., -6.0015, -6.9589, -4.4174]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [27]:
#cv0.597
!cp ../input/incepres15fold320/inceptionres-8-v4-60.pth ./models/
Incepres_learn8 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn8.load('inceptionres-8-v4-60')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds8 = Incepres_learn8.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds8

[tensor([[-4.5106, -4.0405, -5.1868,  ..., -4.4832, -4.2627, -3.9662],
         [-4.1413, -3.6508, -5.1975,  ..., -3.9613, -3.2551, -3.5289],
         [-4.3198, -3.2194, -4.6179,  ..., -3.7789, -3.2199, -3.0611],
         ...,
         [-3.4220, -3.0608, -4.2863,  ..., -3.7475, -3.4194, -3.5976],
         [-1.8856, -4.0043, -4.9988,  ..., -4.5150, -4.2262, -3.5017],
         [-6.2915, -4.7684, -6.6967,  ..., -6.1831, -5.5882, -5.0077]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [28]:
#cv0.588
!cp ../input/incepres15fold320/inceptionres-3-v4-49-n.pth ./models/
Incepres_learn3 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn3.load('inceptionres-3-v4-49-n')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds3 = Incepres_learn3.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds3

[tensor([[-4.9118, -4.1146, -5.3939,  ..., -4.0137, -3.7160, -3.6078],
         [-4.3800, -3.3169, -5.1908,  ..., -3.5141, -3.3256, -3.1718],
         [-4.3927, -3.3530, -5.7068,  ..., -4.4946, -3.7922, -3.6478],
         ...,
         [-3.8732, -3.9473, -5.6140,  ..., -4.9287, -4.0340, -3.8764],
         [-2.4595, -3.7781, -5.4201,  ..., -4.5957, -3.9201, -3.0099],
         [-5.8965, -5.1822, -6.9780,  ..., -5.8385, -6.1045, -4.5300]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [29]:
#cv0.588
!cp ../input/incepres15fold320/inceptionres-7-v4-60-n.pth ./models/
Incepres_learn7 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn7.load('inceptionres-7-v4-60-n')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds7 = Incepres_learn7.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds7

[tensor([[-3.8336, -3.9629, -3.9107,  ..., -3.6296, -3.7494, -3.8698],
         [-3.3321, -2.5065, -4.0323,  ..., -3.3095, -3.1087, -3.5300],
         [-4.4047, -3.0709, -4.9478,  ..., -3.8848, -3.4640, -4.0202],
         ...,
         [-3.3505, -3.7479, -3.9781,  ..., -3.6132, -3.1748, -3.1899],
         [-1.8191, -3.7356, -4.7757,  ..., -4.5177, -3.6311, -3.4725],
         [-6.7270, -5.2728, -7.2506,  ..., -7.1948, -6.3462, -5.7740]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [30]:
#Res_test_preds0

In [31]:
#cv0.594
!cp ../input/incepres15fold320/inceptionres-10-v4-48.pth ./models/
Incepres_learn10 = cnn_learner(Incepres_data, base_arch=myinceptionresnetv2, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Incepres_learn10.load('inceptionres-10-v4-48')
#test_preds = learn0.TTA(ds_type=DatasetType.Test)
Incepres_test_preds10 = Incepres_learn10.get_preds(ds_type=DatasetType.Test)
Incepres_test_preds10

[tensor([[-4.1215, -3.7769, -4.5021,  ..., -3.6988, -3.3641, -3.3777],
         [-4.5454, -3.1572, -4.6904,  ..., -4.0347, -3.7811, -3.5104],
         [-4.2873, -3.3117, -5.2600,  ..., -4.2420, -3.5552, -3.7990],
         ...,
         [-3.7334, -3.2032, -4.2778,  ..., -4.0452, -3.2168, -3.6562],
         [-1.7170, -2.8604, -4.1960,  ..., -3.6264, -3.3653, -2.9614],
         [-6.0555, -5.0829, -6.4952,  ..., -6.0339, -5.6749, -5.1009]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [32]:
!ls ../input/resnet50imet15fold320

resnet50-0-v4-597.pth	resnet50-2-v4-597.pth  resnet50-8-v4-603.pth
resnet50-1-v4-603.pth	resnet50-3-v4-597.pth
resnet50-10-v4-599.pth	resnet50-7-v4-597.pth


In [33]:
#!ls ../input/resnet50-imet-15fold/resnet50-0-v4-35.pth
!cp ../input/resnet50imet15fold320/resnet50-0-v4-597.pth ./models/
Res_learn0 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn0.load('resnet50-0-v4-597')

Res_test_preds0 = Res_learn0.get_preds(ds_type=DatasetType.Test)
Res_test_preds0

[tensor([[-6.8219, -4.9128, -6.2216,  ..., -5.7173, -6.1944, -4.4446],
         [-5.3830, -5.4165, -7.6072,  ..., -5.9351, -6.2061, -5.0721],
         [-5.1742, -3.8567, -5.8140,  ..., -5.5352, -4.7436, -4.5009],
         ...,
         [-3.8951, -4.5864, -5.2852,  ..., -5.1851, -4.4260, -3.7450],
         [-1.9161, -3.3045, -4.8262,  ..., -4.1722, -3.8650, -2.9220],
         [-8.9885, -7.7507, -9.3906,  ..., -8.3082, -9.1043, -6.4195]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [34]:
!cp ../input/resnet50imet15fold320/resnet50-1-v4-603.pth ./models/
Res_learn1 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn1.load('resnet50-1-v4-603')

Res_test_preds1 = Res_learn1.get_preds(ds_type=DatasetType.Test)
Res_test_preds1

[tensor([[-6.9955, -6.4684, -8.8412,  ..., -6.6675, -6.3554, -5.1686],
         [-5.1651, -3.0941, -6.0536,  ..., -4.6852, -4.5963, -4.2565],
         [-4.5970, -2.6846, -4.8918,  ..., -3.8612, -3.3439, -4.0076],
         ...,
         [-4.6550, -5.9307, -6.2103,  ..., -5.5893, -4.8524, -4.2216],
         [-2.1230, -3.1018, -4.7172,  ..., -3.9462, -3.4960, -3.0983],
         [-7.3590, -8.5083, -9.6306,  ..., -7.9608, -7.9215, -7.1964]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [35]:
!cp ../input/resnet50imet15fold320/resnet50-2-v4-597.pth ./models/
Res_learn2 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn2.load('resnet50-2-v4-597')

Res_test_preds2 = Res_learn2.get_preds(ds_type=DatasetType.Test)
Res_test_preds2

[tensor([[-5.7636, -6.2761, -7.1323,  ..., -5.5003, -5.2386, -4.6696],
         [-5.2108, -4.4384, -5.9388,  ..., -5.1696, -4.3296, -3.5728],
         [-6.0074, -3.8978, -7.0067,  ..., -5.4948, -4.4971, -4.6274],
         ...,
         [-4.5869, -4.8818, -6.2680,  ..., -6.3491, -4.5569, -4.3638],
         [-1.6048, -4.2206, -5.7266,  ..., -4.2023, -4.6080, -3.5980],
         [-7.6197, -6.9103, -8.4853,  ..., -7.7118, -7.1763, -6.3087]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [36]:
!cp ../input/resnet50imet15fold320/resnet50-3-v4-597.pth ./models/
Res_learn3 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn3.load('resnet50-3-v4-597')

Res_test_preds3 = Res_learn3.get_preds(ds_type=DatasetType.Test)
Res_test_preds3

[tensor([[-5.3257, -4.5978, -6.1637,  ..., -5.4500, -4.9672, -4.7300],
         [-5.8166, -4.2246, -6.1099,  ..., -5.7708, -5.2559, -4.3212],
         [-5.8628, -3.9773, -6.7349,  ..., -5.7162, -4.3297, -4.8485],
         ...,
         [-3.8142, -3.6804, -4.5864,  ..., -4.7170, -3.9138, -3.8047],
         [-2.6364, -4.1125, -4.9840,  ..., -4.5171, -4.7519, -3.6889],
         [-9.5973, -7.3158, -9.8958,  ..., -9.4122, -8.3396, -7.0970]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [37]:
!cp ../input/resnet50imet15fold320/resnet50-7-v4-597.pth ./models/
Res_learn7 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn7.load('resnet50-7-v4-597')

Res_test_preds7 = Res_learn7.get_preds(ds_type=DatasetType.Test)
Res_test_preds7

[tensor([[-5.9335, -4.8796, -6.5213,  ..., -6.2499, -5.5430, -4.7291],
         [-7.0723, -4.7764, -7.4052,  ..., -6.8222, -5.9230, -4.4368],
         [-6.4689, -3.6933, -7.7250,  ..., -6.9396, -5.7691, -4.8374],
         ...,
         [-3.7146, -3.5165, -4.7874,  ..., -4.7528, -3.6526, -3.9040],
         [-2.1136, -4.2081, -4.9279,  ..., -5.0937, -4.5780, -3.5033],
         [-8.4585, -6.5192, -9.2277,  ..., -9.1834, -8.2336, -6.8256]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [38]:
!cp ../input/resnet50imet15fold320/resnet50-8-v4-603.pth ./models/
Res_learn8 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn8.load('resnet50-8-v4-603')

Res_test_preds8 = Res_learn8.get_preds(ds_type=DatasetType.Test)
Res_test_preds8

[tensor([[-5.5920, -5.3055, -6.8685,  ..., -5.4316, -5.9578, -5.4114],
         [-6.7272, -5.3598, -7.6389,  ..., -6.9747, -6.2627, -5.6240],
         [-4.9920, -3.2650, -5.8382,  ..., -4.9030, -3.8423, -3.9319],
         ...,
         [-4.0514, -3.9973, -5.2370,  ..., -5.0518, -4.2274, -3.7888],
         [-2.4471, -4.1715, -5.9067,  ..., -4.9435, -5.5388, -3.9696],
         [-7.7245, -6.0468, -9.2368,  ..., -7.3863, -7.2527, -6.3081]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [39]:
!cp ../input/resnet50imet15fold320/resnet50-10-v4-599.pth ./models/
Res_learn10 = cnn_learner(Res_data, base_arch=models.resnet50, loss_func=FocalLoss(), metrics=fbeta,pretrained=False)
Res_learn10.load('resnet50-10-v4-599')

Res_test_preds10 = Res_learn10.get_preds(ds_type=DatasetType.Test)
Res_test_preds10

[tensor([[-5.4756, -6.3119, -6.7072,  ..., -6.6551, -5.7700, -5.3237],
         [-5.3487, -3.2242, -6.0722,  ..., -5.3025, -3.9491, -4.0745],
         [-4.5993, -2.6871, -5.3087,  ..., -4.4860, -3.4799, -3.5989],
         ...,
         [-3.8987, -3.3031, -5.2421,  ..., -5.1105, -3.8502, -3.7927],
         [-2.3614, -3.9477, -5.3300,  ..., -4.4986, -4.2629, -3.6184],
         [-7.8013, -6.2088, -8.9449,  ..., -6.9325, -7.7969, -5.9081]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [40]:
#import numpy as np
#np.shape(np.array(Res_test_preds0[0]))


In [41]:
test_preds = ((Incepres_test_preds0[0]*1+Incepres_test_preds1[0]*1.1
               +Incepres_test_preds2[0]*0.9+Incepres_test_preds3[0]*0.9
               +Incepres_test_preds7[0]*0.9+Incepres_test_preds8[0]*1.1
               +Incepres_test_preds10[0]*1.1)/16+(Res_test_preds0[0]*1+Res_test_preds1[0]*1
               +Res_test_preds2[0]*1+Res_test_preds3[0]*1
               +Res_test_preds7[0]*1+Res_test_preds8[0]*1
               +Res_test_preds10[0]*1)/16+
              (Sex_test_preds3[0]*0.5+Sex_test_preds2[0]*0.5+Sex_test_preds1[0]*0.5+Sex_test_preds0[0]*0.5)/16,Incepres_test_preds1[1])
test_preds
#test_preds = ((test_preds0[0]+test_preds1[0]+test_preds2[0]+test_preds3[0]+test_preds7[0]+test_preds8[0]+test_preds10[0])/7,test_preds1[1])

(tensor([[-5.1517, -4.6520, -5.6969,  ..., -4.8940, -4.6947, -4.2797],
         [-5.1427, -3.8092, -5.8082,  ..., -4.8835, -4.4074, -3.9875],
         [-4.9437, -3.3104, -5.6644,  ..., -4.7058, -4.0012, -4.0280],
         ...,
         [-3.9246, -3.9178, -5.0954,  ..., -4.7322, -3.8815, -3.7979],
         [-2.0228, -3.7733, -5.0058,  ..., -4.3733, -4.1843, -3.3164],
         [-7.1718, -5.9697, -7.8145,  ..., -6.9571, -6.7874, -5.6647]]),
 tensor([0, 0, 0,  ..., 0, 0, 0]))

In [42]:
i2c = np.array([[i, c] for c, i in Incepres_learn0.data.train_ds.y.c2i.items()]).astype(int)
def join_preds(preds, thr):
    return [' '.join(i2c[np.where(t==1)[0],1].astype(str)) for t in (preds[0].sigmoid()>thr).long()]

In [43]:
test_df.attribute_ids = join_preds(test_preds, 0.270)
test_df.head()

,id,attribute_ids
0,10023b2cc4ed5f68,1059 195 223 766
1,100fbe75ed8fd887,1039 872
2,101b627524a04f19,420 784 79
3,10234480c41284c6,1046 147 480 483 725 738 776 813 830 923
4,1023b0e2636dcea8,1046 1092 147 322 477 584 671 813 954


In [44]:
test_df.to_csv('submission.csv', index=False) 